In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

DATAFILE = "data/Leoni.dat"
df = pd.read_csv(DATAFILE, sep=";")

y = df["Schlusskurs"].values
t = df["Eintrag"].values
d = pd.to_datetime(df["Datum"], format="%d.%m.%Y")
u = np.zeros(len(y))

Ts = t[1] - t[0]

print(Ts)


1


In [211]:
y_rand_var = []
for i in range(0, 3, 1):
    y_rand_temp = random.choices(y[0:60], k=30)

    y_rand_var.append(np.var(y_rand_temp))
    # y_rand_mean = np.mean(y_rand_temp)
    # y_rand_var_temp = np.var(y_rand_temp)
    # y_rand_var = y_rand_var_temp if y_rand_var_temp < y_rand_var else y_rand_var
    # if (y_rand_var_temp < y_rand_var):
    #     y_rand_var = y_rand_var_temp

R = np.eye(2) * min(y_rand_var)
print(R)
Q = np.eye(2) * 1e-4


0.9256112222222221


In [212]:
# Define system matrices
Ad = np.array([[1, Ts], [0, 1]])
Bd = np.array([[0], [0]])
C = np.array([[1, 0]])
D = np.array([0])
Gd = np.array([[Ts], [1]])

# Initialization
x_dach = np.array([y[0], 0])
P_dach = 50 * np.eye(2)

print(Gd)


[[1]
 [1]]


In [213]:
# Kalman filter
s = np.zeros(len(y))
v = np.zeros(len(y))
K1 = np.zeros(len(y))
K2 = np.zeros(len(y))
P_tilde1 = np.zeros(len(y))
P_tilde2 = np.zeros(len(y))
P_tilde3 = np.zeros(len(y))
P_tilde4 = np.zeros(len(y))
d_y = np.zeros(len(y))

for k in range(len(y)):
    # print((C @ x_dach + D[k] * u[k]))
    # print((D @ u[k]))
    # print(y[k] - (C @ x_dach + D * u[k]))
    # d_y[k] = y[k] - (C @ x_dach + D @ u[k])
    d_y[k] = y[k] - (C @ x_dach + D[k] * u[k])
    K = P_dach @ C.T @ np.linalg.pinv(C @ P_dach @ C.T + R)
    # x_tilde = x_dach + K @ d_y[k]
    # print((K @ d_y.reshape(1, -1)).shape)
    # print(x_dach)
    x_tilde = x_dach[0] + K @ d_y.reshape(1, -1)
    P_tilde = (np.eye(len(Bd)) - K @ C) @ P_dach
    x_dach = Ad @ x_tilde + Bd @ u[k].reshape(-1, 1)
    # P_dach = Ad @ P_tilde @ Ad.T + Gd @ Q @ Gd.T

    print(x_tilde[1])

    P_dach = Ad @ P_tilde @ Ad.T + Gd.T @ Q @ Gd
    s[k] = x_tilde[0][0]
    v[k] = x_tilde[1][0]
    K1[k] = K[0][0]
    K2[k] = K[1][0]
    P_tilde1[k] = P_tilde[0][0]
    P_tilde2[k] = P_tilde[0][1]
    P_tilde3[k] = P_tilde[1][0]
    P_tilde4[k] = P_tilde[1][1]

# Plot results
plt.figure(1)
plt.subplot(2, 1, 1)
plt.plot(
    t,
    y,
    "k*",
    t,
    s,
    "b--",
    t,
    s + 1.5 * np.sqrt(P_tilde1),
    "b-",
    t,
    s - 1.5 * np.sqrt(P_tilde1),
    "b-",
)
plt.grid(True)
plt.xlabel("Zeit / Tag")
plt.ylabel("Kurs / Euro")
plt.subplot(2, 1, 2)
plt.plot(t, v, "r-*")
plt.grid(True)
plt.xlabel("Zeit / Tag")
plt.ylabel("Kursänderungen / Euro/Tag")
plt.show()


[29.87 29.87 29.87 ... 29.87 29.87 29.87]


IndexError: index 1 is out of bounds for axis 0 with size 1